In [1]:
#!pip install dendropy
#!/usr/bin/python
import dendropy
from pandas import Series, DataFrame
import numpy as np
import function_1

In [2]:
#import dendropy
#from pandas import Series, DataFrame
#import numpy as np


def make_distance_matrix(distances, sort=False):
    distances = distances.reset_index().pivot_table(index=0, columns=1, values='distance')
    distances.index.name=None
    distances.columns.name=None
    distances = distances.fillna(distances.T)

    missing_rows = [i for i in distances.index if i not in distances.columns]
    for row in missing_rows:
        distances[row] = distances.loc[row]

    missing_columns = [c for c in distances.columns if c not in distances.index]
    for column in missing_columns:
        distances.loc[column] = distances[column]
    
    if sort:
        distances = distances.sort_index().sort_index(axis=1)
    return distances.fillna(0)



def d(start, end, distances):
    return distances.loc[(start, end), 'distance']



def get_distance_dataframe(tree):
    pdm = tree.phylogenetic_distance_matrix()
    distances = [[*name, distance] for name, distance in zip(pdm.distinct_taxon_pair_iter(), pdm.distances())]
    distances = DataFrame(distances)
    #print(distances)
    
    def get_label(node):
        return node.label
    
    distances[0] = distances[0].map(get_label)
    distances[1] = distances[1].map(get_label)
    
    distances = distances.set_index([0, 1])
    distances = distances.rename(columns = {2:'distance'})

    return distances


def get_distance_topology(tree):
    pdm = tree.phylogenetic_distance_matrix()
    distances = [[*name, distance] for name, distance in zip(pdm.distinct_taxon_pair_iter(), pdm.distances())]
    

    for pairs in distances:
        pairs[2] = pdm.path_edge_count(taxon1 = pairs[0], taxon2 = pairs[1])
    
    distances = DataFrame(distances)
    def get_label(node):
        return node.label
    
    distances[0] = distances[0].map(get_label)
    distances[1] = distances[1].map(get_label)
    distances = distances.set_index([0, 1])
    distances = distances.rename(columns = {2:'distance'})
    #distances = DataFrame(distances)
    
    #print("hello")
    return distances



def get_ratio(distances1, distances2):
    r1 = distances1.loc[common_leaves, common_leaves].sum().sum() / 2
    r2 = distances2.loc[common_leaves, common_leaves].sum().sum() / 2
    return r1/r2

def noisify_distances(distances):
    noise = np.triu(1 + np.random.random(distances.shape) / 10).round(2)
    noise += noise.T
    return distances * noise


#def temp_paralog(taxalist):
    


def get_paralogs(taxa_list1, taxa_list2):
    
    common_leaves = []
    temp_leaves = []
    paralogs_species = []
    dico_identifier = {}
    
    CommonLeaves = {}
    Paralogs = {}
    
    for c in taxa_list1:
        #This splitting method is not ideal : might not always have the same way to separate specie and gene -> add verification when launching program ?
        taxon = c.split(' ')[0]
        gene = c.split(' ')[1]
        
        if taxon in dico_identifier.keys() :
            dico_identifier[taxon].append(gene)
        else : dico_identifier[taxon] = [gene]

        if taxon in temp_leaves :
            paralogs_species.append(taxon)
            temp_leaves.remove(taxon)
            
        else :
            temp_leaves.append(taxon)

    
    for d in taxa_list2:
        taxon = d.split(' ')[0]
        gene = d.split(' ')[1]
        
        if taxon in dico_identifier.keys() :
            dico_identifier[taxon].append(gene)
        else : dico_identifier[taxon] = [gene]
        
        if taxon in temp_leaves :
            common_leaves.append(taxon)
            temp_leaves.remove(taxon)
            
        elif taxon in common_leaves :
            paralogs_species.append(taxon)
            common_leaves.remove(taxon)
    
    for specie in paralogs_species : Paralogs[specie] = dico_identifier[specie]
    for common in common_leaves : CommonLeaves[common] = dico_identifier[common]
    
    return Paralogs, CommonLeaves
            

In [3]:
def get_paralogs_variant(taxa1, taxa2, common_leaves, paralogs):
    
    dict_T1 = {"Specie":[], "Identifier":[], "Alias":[], "Distance":[]}
    dict_T2 = {"Specie":[], "Identifier":[], "Alias":[], "Distance":[]}
    
    for c in taxa1:
        taxon = c.split(' ')[0]
        gene = c.split(' ')[1]
        dict_T1["Specie"].append(taxon)
        dict_T1["Identifier"].append(gene)
        dict_T1["Alias"].append('')
        dict_T1["Distance"].append(0)
    
    for d in taxa2:
        taxon = d.split(' ')[0]
        gene = d.split(' ')[1]
        dict_T2["Specie"].append(taxon)
        dict_T2["Identifier"].append(gene)
        dict_T2["Alias"].append('')
        dict_T2["Distance"].append(0)
        
    temp_common = np.intersect1d(dict_T1["Specie"],dict_T2["Specie"])
    for element in temp_common:
        if dict_T1["Specie"].count(element) == 1 and dict_T2["Specie"].count(element) == 1:
            common_leaves.append(element)
        else : paralogs.append(element)
    
    return dict_T1, dict_T2

In [4]:
tree1 = dendropy.Tree.get(path='sim_tree_TOPO1', schema='newick', suppress_edge_lengths = False)
# or whatever relevant format if not newick
tree2 = dendropy.Tree.get(path='sim_tree_TOPO2', schema='newick', suppress_edge_lengths = False)



In [5]:
print(tree1.as_ascii_plot())
#essai = tree.phylogenetic_distance_matrix().distinct_taxon_pair_iter()
#essai
#tree.taxon()

                                                /------------------------ 9 b 
                        /-----------------------+                             
/-----------------------+                       \------------------------ 50 x
|                       |                                                     
|                       \------------------------------------------------ 12 b
+                                                                             
|                                               /------------------------ 9 a 
|                       /-----------------------+                             
|                       |                       \------------------------ 12 a
\-----------------------+                                                     
                        |                       /------------------------ 67 x
                        \-----------------------+                             
                                                \---

In [6]:
#print(tree.as_ascii_plot())
list_name = []
for name in tree.phylogenetic_distance_matrix().distinct_taxon_pair_iter():
    if name[0] not in list_name :
        list_name.append(name[0])
    elif name[1] not in list_name :
        list_name.append(name[1])
        #print(name)
        
        
len(list_name)
list_name[:2]

NameError: name 'tree' is not defined

In [6]:
distances1 = get_distance_topology(tree1)
distances2 = get_distance_topology(tree2)
#distances1
#distances2

In [8]:
distances1

distance
0    1             
50 x 12 a         6
9 b  50 x         2
9 a  12 a         2
     67 x         4
12 a 42 x         4
9 b  12 a         6
12 b 9 a          5
9 b  42 x         6
12 b 42 x         5
50 x 9 a          6
12 a 67 x         4
9 b  67 x         6
     12 b         3
9 a  42 x         4
50 x 12 b         3
     67 x         6
67 x 42 x         2
50 x 42 x         6
9 b  9 a          6
12 b 12 a         5
     67 x         5

In [10]:
test = make_distance_matrix(distances1, True)
taxon = test.columns[0]
taxa = test.columns
taxa

Index(['12 a', '12 b', '42 x', '50 x', '67 x', '9 a', '9 b'], dtype='object')

In [12]:
test['12 a']['9 a']

2.0

In [53]:
common_leaves = {}
CL = []
paralogs = {}
para = []
paralogs, common_leaves = get_paralogs_variant(taxa, taxa, CL, para)
#common_leaves = common_leaves[:5]

#print(common_leaves)
paralogs
tired = DataFrame(paralogs, index = (paralogs['Specie']))

In [55]:
tired.loc['12']

,Specie,Identifier,Alias,Distance
12,12,a,,0
12,12,b,,0


In [57]:
tired = DataFrame(paralogs)
tired[tired['Specie'] == '12']
#tired.replace(tired['Specie'][0], 5, regex = False)



,Specie,Identifier,Alias,Distance
0,12,a,,0
1,12,b,,0


In [152]:
a = ['n212', 'n257', 'n271', 'n304', 'n316', 'n317']
b = ['n1', 'n102', 'n202', 'n211', 'n214']


In [58]:
tired[tired['Specie'] == '12'][tired['Identifier']=='b']

C:\Users\annaa\AppData\Local\Temp\ipykernel_21816\4002978614.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tired[tired['Specie'] == '12'][tired['Identifier']=='b']


,Specie,Identifier,Alias,Distance
1,12,b,,0


In [32]:
tree1 = dendropy.Tree.get(path='tree_test.txt', schema='newick') # or whatever relevant format if not newick
distances1 = get_distance_dataframe(tree1)
distances1 = make_distance_matrix(distances1, True)
distances1 = noisify_distances(distances1)

tree2 = dendropy.Tree.get(path='tree_test2.txt', schema='newick') # or whatever relevant format if not newick
distances2 = get_distance_dataframe(tree2)
distances2 = make_distance_matrix(distances2, True)


In [33]:
common_leaves = ['A', 'C', 'G']
# 'B'->'X', 'D'->'R', 'F'->'W', 'H'->'Z'
common_leaves = [c for c in distances2.columns if c in distances1.columns]
epsilon = 0
common_leaves

['A', 'C', 'G']

In [17]:
def baseline_method(distances1, distances2, epsilon, common_leaves):
    # step 1 compute ratio    
    R = get_ratio(distances1, distances2)
    print('ratio:', R)
    
    # step 2 compute distances from candidate to common leaves
    candidates2 = [c for c in distances2.columns if c not in common_leaves]
    candidate_distances2 = distances2.loc[candidates2, common_leaves].sum(axis=1)
    
    candidates1 = [c for c in distances1.columns if c not in common_leaves]
    candidate_distances1 = distances1.loc[candidates1, common_leaves].sum(axis=1)
    
    # step 3 compare distances adjusted with the ratio
    candidate_distances2 = candidate_distances2 * R
    
    # step 4 find the closest matches
    matches = {}
    for candidate in candidates2:
        d_candidate = candidate_distances2[candidate]
        sorted_matches = (candidate_distances1 - d_candidate).abs().sort_values()        
        matches[candidate] = sorted_matches[sorted_matches <= sorted_matches.min() + epsilon].to_dict()

    return DataFrame(matches).stack().dropna().sort_values()

In [9]:
baseline_method(distances1, distances2, .5, common_leaves).sort_index()

ratio: 1.0569117647058823


B  R    0.531853
   X    0.003397
D  R    0.076853
   X    0.451603
   Z    0.288235
E  W    0.150529
   Y    0.008007
F  W    0.055971
   Y    0.214507
H  R    0.140147
   Z    0.071235
dtype: float64

In [10]:
common_leaves

['A', 'C', 'G']

In [11]:
candidates1 = [c for c in distances1.columns if c not in common_leaves]
distances1.loc[candidates1, common_leaves].sum(axis=1).sort_values()

B    2.4275
D    2.8825
H    3.0995
E    4.2885
F    4.4950
dtype: float64

In [12]:
candidates2 = [c for c in distances2.columns if c not in common_leaves]
distances2.loc[candidates2, common_leaves].sum(axis=1).sort_values()

X    2.30
R    2.80
Z    3.00
Y    4.05
W    4.20
dtype: float64

In [34]:
# 'B'->'X', 'D'->'R', 'F'->'W', 'H'->'Z', 'E' -> 'Y'

In [ ]:
# Try correlation method
# Try stats
# Try ordering common leaf by distances from paralogs -> moyen de décision pour groupe de paralogues ou paralogues
#     avec des distances égales/proches => pour quand on souhaite garder une topologie similaire